In [39]:
import numpy as np
import pandas as pd

In [40]:
from google.colab import files
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [41]:
df_ques_cluster = pd.read_csv('/drive/My Drive/Research Project IIT-I/AP Question clusters.csv')

In [42]:
df_ques_cluster

,index,BlockName,Category,Year,Month,Day,Crop,DistrictName,QueryType,Sector,StateName,QueryText,KccAns,Cluster
0,23781986,SARAVAKOTA,Others,2020,6,10,Others,SRIKAKULAM,Government Schemes,AGRICULTURE,ANDHRA PRADESH,MAO NUMBER,-- RECOMMENDED TO CONTACT 8886612698,1
1,23824164,NELLORE,Others,2020,3,17,Others,NELLORE,Seeds,AGRICULTURE,ANDHRA PRADESH,MAO NUMBER,-- 8886613101,1
2,23824118,KODAVALUR,Green Manure,2020,3,16,KolanchiTephrosia Purpurea,NELLORE,Varieties,AGRICULTURE,ANDHRA PRADESH,MAO NUMBER,8886613686,1
3,23824098,MARRIPADU,Others,2020,3,15,Others,NELLORE,Government Schemes,AGRICULTURE,ANDHRA PRADESH,MAO NUMBER,8886613671,1
4,23821955,MANUBOLU,Others,2020,1,25,Others,NELLORE,Government Schemes,AGRICULTURE,ANDHRA PRADESH,MAO NUMBER,MAO NUMBER8886613708,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26714,23825672,CHILLAKUR,Oilseeds,2020,10,23,Groundnut pea nutmung phalli,NELLORE,Seeds,AGRICULTURE,ANDHRA PRADESH,GROUNDS NUT VARIETY,-- RECOMMENDED TO CONTACT 9989628385,568
26715,23825977,VENKATAGIRI,Oilseeds,2020,11,12,Groundnut pea nutmung phalli,NELLORE,Seeds,AGRICULTURE,ANDHRA PRADESH,kadiri ground nut varieties,Agricultural Research Station Kadiri - 515 591...,568
26716,23823419,RAPUR,Fruits,2020,2,25,Citrus,NELLORE,Agriculture Mechanization,HORTICULTURE,ANDHRA PRADESH,CITRUS Scab MANAGEMENT,recommended to spray amistar top azoxystrobind...,569
26717,23824455,BALAYAPALLE,Fruits,2020,7,6,Citrus,NELLORE,Plant Protection,HORTICULTURE,ANDHRA PRADESH,CITRUS SCAB MANAGEMENT,RECOMMENDED TO SPRAY CARBENDAZIM BAVISTIN 2GM ...,569


In [43]:
import nltk
nltk.download('punkt')
from gensim.parsing.preprocessing import remove_stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [44]:
ps = PorterStemmer()

In [45]:
def remove_special_char(string):
  special_characters=[',', '?', '.']
  normal_string = "".join(filter(lambda char: char not in special_characters , string))
  return normal_string

def clean_query(query):
  query = query.lower()
  query = query.replace('-', ' ')
  query = remove_special_char(query)
  query = remove_stopwords(query)

  processed_tokens = []
  for word in word_tokenize(query):
    if not word.isnumeric():
      processed_tokens.append(ps.stem(word))
  return ' '.join(processed_tokens)

In [46]:
processed_queries = df_ques_cluster.QueryText.apply(clean_query)

In [47]:
processed_queries

0                       mao number
1                       mao number
2                       mao number
3                       mao number
4                       mao number
                   ...            
26714           ground nut varieti
26715    kadiri ground nut varieti
26716             citru scab manag
26717             citru scab manag
26718             citru scab manag
Name: QueryText, Length: 26719, dtype: object

In [48]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
CV = CountVectorizer()

In [49]:
vectors = CV.fit_transform(processed_queries).toarray()

In [50]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [51]:
vectors.shape

(26719, 1103)

In [52]:
labels = pd.get_dummies(df_ques_cluster.Cluster)

In [53]:
labels

,1,2,3,4,5,6,7,8,9,10,...,560,561,562,563,564,565,566,567,568,569
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26714,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
26715,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
26716,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
26717,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [54]:
from sklearn.model_selection import train_test_split

In [55]:
X_train, X_test, y_train, y_test = train_test_split(vectors, labels, test_size = 0.2, random_state = 42)

In [56]:
X_train.shape

(21375, 1103)

In [57]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [58]:
from sklearn import svm

In [59]:
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X_train, np.array([(np.argmax(x)+1) for x in np.array(y_train)]))

SVC(decision_function_shape='ovo')

In [60]:
y_pred = clf.predict(X_test)

In [61]:
y_pred

array([ 78,   5,  66, ..., 107,  64, 287])

In [62]:
actual = [(np.argmax(x)+1) for x in np.array(y_test)]
actual = np.array(actual)

In [63]:
actual

array([ 78,   5,  66, ..., 107,  64, 287])

In [64]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

print('weighted', precision_recall_fscore_support(actual, y_pred, average='weighted'))
print('micro',precision_recall_fscore_support(actual, y_pred, average='micro'))
print('macro',precision_recall_fscore_support(actual, y_pred, average='macro'))
print('accuracy', accuracy_score(actual, y_pred))

weighted (0.93687846931226, 0.9541541916167665, 0.942717472003819, None)
micro (0.9541541916167665, 0.9541541916167665, 0.9541541916167665, None)
macro (0.6920111520876271, 0.7057127734014181, 0.6890307431650493, None)
accuracy 0.9541541916167665


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1